In [1]:
import selenium
from selenium import webdriver
import time
from PIL import Image
import requests
import os
import io
import hashlib

In [2]:
requests.session().keep_alive = False
requests.adapters.DEFAULT_RETRIES = 5

In [3]:
ROOT_DIR = '/home/ayman/FOE-Linux/Task/Day-Night-Classifier/scraped_dataset'
DAY_IMGS = os.path.join(ROOT_DIR, 'day')
NIGHT_IMGS = os.path.join(ROOT_DIR, 'night')

In [15]:
class ScrapeImage:
    def __init__(self, wd:webdriver, sleep_between_interactions:float=1):
        self.wd = wd
        self.sleep = sleep_between_interactions
    
    def fetch_image_urls(self, query:str, max_links_to_fetch:int):
        
        # build the google query
        search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"
        
         # load the page
        self.wd.get(search_url.format(q=query))
        
        image_urls = set()
        image_count = 0
        results_start = 0
        
        while image_count < max_links_to_fetch:
            self.__scroll_to_end(self.wd)

            # get all image thumbnail results
            thumbnail_results = self.wd.find_elements_by_css_selector("img.Q4LuWd")
            number_results = len(thumbnail_results)

            print(f"Found: {number_results} search results. Extracting links from {results_start}:{number_results}")

            for img in thumbnail_results[results_start:number_results]:
                # try to click every thumbnail such that we can get the real image behind it
                try:
                    img.click()
                    time.sleep(self.sleep)
                except Exception:
                    continue

                # extract image urls    
                actual_images = self.wd.find_elements_by_css_selector('img.n3VNCb')
                for actual_image in actual_images:
                    if actual_image.get_attribute('src') and 'https' in actual_image.get_attribute('src'):
                        image_urls.add(actual_image.get_attribute('src'))

                image_count = len(image_urls)

                if len(image_urls) >= max_links_to_fetch:
                    print(f"Found: {len(image_urls)} image links, done!")
                    break
                else:
                    print("Found:", len(image_urls), "image links, looking for more ...")
                    time.sleep(self.sleep)
                    load_more_button = self.wd.find_element_by_css_selector(".mye4qd")
                    if load_more_button:
                        self.wd.execute_script("document.querySelector('.mye4qd').click();")

                # move the result startpoint further down
                results_start = len(thumbnail_results)

        return image_urls
    
    def __scroll_to_end(self, wd):
        self.wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(self.sleep)    
        
    def download_image(self, folder_path:str, url:str):
        try:
            image_content = requests.get(url).content

        except Exception as e:
            print(f"ERROR - Could not download {url} - {e}")

        try:
            image_file = io.BytesIO(image_content)
            image = Image.open(image_file).convert('RGB')
            # Assigning the image a random 10-digit id.
            file_path = os.path.join(folder_path, hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
            
            with open(file_path, 'wb') as f:
                image.save(f, "JPEG", quality=100, subsampling=0)
            print(f"SUCCESS - saved {url} - as {file_path}")
            
        except Exception as e:
            print(f"ERROR - Could not save {url} - {e}")
            

In [16]:
wd = webdriver.Firefox()
scrape = ScrapeImage(wd, 0.005)

In [6]:
sub_text = ['night in ', 'sunny day in '] 
queries = ['norway', 'germany', 'france', 'armenia', 'austria', 'italy', 'belgium', 'denmark',
          'finland', 'portugal', 'netherlands', 'united kingdom', 'sydney']
len(queries)

13

In [109]:
urls = {0: [], 1: []}
for i in range(2):
    for query in queries:
        out = list(scrape.fetch_image_urls(sub_text[i] + query, 50))
        for url in out:
            print(url)
            urls[i].append(url)
        print(query + ' Done!')
    print(f"{i}s Category is done")

Found: 100 search results. Extracting links from 0:100
Found: 0 image links, looking for more ...
Found: 0 image links, looking for more ...
Found: 1 image links, looking for more ...
Found: 2 image links, looking for more ...
Found: 3 image links, looking for more ...
Found: 3 image links, looking for more ...
Found: 3 image links, looking for more ...
Found: 3 image links, looking for more ...
Found: 3 image links, looking for more ...
Found: 3 image links, looking for more ...
Found: 3 image links, looking for more ...
Found: 3 image links, looking for more ...
Found: 3 image links, looking for more ...
Found: 3 image links, looking for more ...
Found: 3 image links, looking for more ...
Found: 3 image links, looking for more ...
Found: 3 image links, looking for more ...
Found: 3 image links, looking for more ...
Found: 3 image links, looking for more ...
Found: 4 image links, looking for more ...
Found: 5 image links, looking for more ...
Found: 6 image links, looking for more ...

Found: 0 image links, looking for more ...
Found: 0 image links, looking for more ...
Found: 0 image links, looking for more ...
Found: 0 image links, looking for more ...
Found: 1 image links, looking for more ...
Found: 1 image links, looking for more ...
Found: 1 image links, looking for more ...
Found: 1 image links, looking for more ...
Found: 1 image links, looking for more ...
Found: 1 image links, looking for more ...
Found: 2 image links, looking for more ...
Found: 3 image links, looking for more ...
Found: 4 image links, looking for more ...
Found: 5 image links, looking for more ...
Found: 6 image links, looking for more ...
Found: 7 image links, looking for more ...
Found: 8 image links, looking for more ...
Found: 9 image links, looking for more ...
Found: 10 image links, looking for more ...
Found: 11 image links, looking for more ...
Found: 12 image links, looking for more ...
Found: 13 image links, looking for more ...
Found: 14 image links, looking for more ...
Found:

Found: 3 image links, looking for more ...
Found: 3 image links, looking for more ...
Found: 3 image links, looking for more ...
Found: 3 image links, looking for more ...
Found: 4 image links, looking for more ...
Found: 5 image links, looking for more ...
Found: 6 image links, looking for more ...
Found: 7 image links, looking for more ...
Found: 8 image links, looking for more ...
Found: 9 image links, looking for more ...
Found: 10 image links, looking for more ...
Found: 11 image links, looking for more ...
Found: 12 image links, looking for more ...
Found: 13 image links, looking for more ...
Found: 14 image links, looking for more ...
Found: 15 image links, looking for more ...
Found: 16 image links, looking for more ...
Found: 17 image links, looking for more ...
Found: 18 image links, looking for more ...
Found: 19 image links, looking for more ...
Found: 21 image links, looking for more ...
Found: 22 image links, looking for more ...
Found: 23 image links, looking for more ..

Found: 13 image links, looking for more ...
Found: 14 image links, looking for more ...
Found: 15 image links, looking for more ...
Found: 17 image links, looking for more ...
Found: 18 image links, looking for more ...
Found: 20 image links, looking for more ...
Found: 21 image links, looking for more ...
Found: 23 image links, looking for more ...
Found: 24 image links, looking for more ...
Found: 25 image links, looking for more ...
Found: 26 image links, looking for more ...
Found: 27 image links, looking for more ...
Found: 28 image links, looking for more ...
Found: 29 image links, looking for more ...
Found: 30 image links, looking for more ...
Found: 31 image links, looking for more ...
Found: 32 image links, looking for more ...
Found: 33 image links, looking for more ...
Found: 34 image links, looking for more ...
Found: 35 image links, looking for more ...
Found: 36 image links, looking for more ...
Found: 37 image links, looking for more ...
Found: 38 image links, looking f

Found: 29 image links, looking for more ...
Found: 30 image links, looking for more ...
Found: 32 image links, looking for more ...
Found: 34 image links, looking for more ...
Found: 35 image links, looking for more ...
Found: 36 image links, looking for more ...
Found: 37 image links, looking for more ...
Found: 38 image links, looking for more ...
Found: 40 image links, looking for more ...
Found: 41 image links, looking for more ...
Found: 42 image links, looking for more ...
Found: 43 image links, looking for more ...
Found: 44 image links, looking for more ...
Found: 45 image links, looking for more ...
Found: 46 image links, looking for more ...
Found: 47 image links, looking for more ...
Found: 48 image links, looking for more ...
Found: 49 image links, looking for more ...
Found: 50 image links, done!
https://thumbs.dreamstime.com/b/classic-view-hallstatt-village-austria-night-201514056.jpg
https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSgdLbj6dK-nOl-pFgrI9RX9MvFwCXJgAec

...

Found: 0 image links, looking for more ...
Found: 1 image links, looking for more ...
Found: 2 image links, looking for more ...
Found: 3 image links, looking for more ...
Found: 3 image links, looking for more ...
Found: 3 image links, looking for more ...
Found: 3 image links, looking for more ...
Found: 3 image links, looking for more ...
Found: 3 image links, looking for more ...
Found: 3 image links, looking for more ...
Found: 3 image links, looking for more ...
Found: 4 image links, looking for more ...
Found: 6 image links, looking for more ...
Found: 7 image links, looking for more ...
Found: 9 image links, looking for more ...
Found: 10 image links, looking for more ...
Found: 12 image links, looking for more ...
Found: 14 image links, looking for more ...
Found: 16 image links, looking for more ...
Found: 17 image links, looking for more ...
Found: 18 image links, looking for more ...
Found: 20 image links, looking for more ...
Found: 22 image links, looking for more ...
Fou

Found: 18 image links, looking for more ...
Found: 19 image links, looking for more ...
Found: 20 image links, looking for more ...
Found: 21 image links, looking for more ...
Found: 22 image links, looking for more ...
Found: 23 image links, looking for more ...
Found: 24 image links, looking for more ...
Found: 25 image links, looking for more ...
Found: 26 image links, looking for more ...
Found: 27 image links, looking for more ...
Found: 28 image links, looking for more ...
Found: 29 image links, looking for more ...
Found: 30 image links, looking for more ...
Found: 31 image links, looking for more ...
Found: 32 image links, looking for more ...
Found: 33 image links, looking for more ...
Found: 34 image links, looking for more ...
Found: 35 image links, looking for more ...
Found: 36 image links, looking for more ...
Found: 37 image links, looking for more ...
Found: 38 image links, looking for more ...
Found: 39 image links, looking for more ...
Found: 40 image links, looking f

Found: 24 image links, looking for more ...
Found: 25 image links, looking for more ...
Found: 26 image links, looking for more ...
Found: 27 image links, looking for more ...
Found: 28 image links, looking for more ...
Found: 29 image links, looking for more ...
Found: 30 image links, looking for more ...
Found: 31 image links, looking for more ...
Found: 32 image links, looking for more ...
Found: 33 image links, looking for more ...
Found: 35 image links, looking for more ...
Found: 37 image links, looking for more ...
Found: 39 image links, looking for more ...
Found: 41 image links, looking for more ...
Found: 42 image links, looking for more ...
Found: 44 image links, looking for more ...
Found: 45 image links, looking for more ...
Found: 46 image links, looking for more ...
Found: 48 image links, looking for more ...
Found: 49 image links, looking for more ...
Found: 50 image links, done!
https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQQ4DdxWmT3SYonDlp7bu40iVZckNk7sf2fgQ&

Found: 50 image links, done!
https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTjoUhEcpHKpWpGIHZYsub3Y3ZsOkK61O7-oA&usqp=CAU
https://images.snapwi.re/bb5d/5ed2d7310cfbeb0a1f51c601.w800.jpg
https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSvSI46uZMVMf2Bqqq83_eRyBGCyVHwNfb_Kg&usqp=CAU
https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTNaro1B00oaiWeY4xMzscjXEqhhfC_YMZFfQ&usqp=CAU
https://st3.depositphotos.com/3411843/18273/i/1600/depositphotos_182735070-stock-photo-amsterdam-capital-of-netherlands-on.jpg
https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcR_pMKsSVXRSdXFfnAYcEj4uGTBTtngnhIHzQ&usqp=CAU
https://upload.wikimedia.org/wikipedia/commons/0/0e/Vondelpark_On_A_Sunny_Day_Amsterdam_Holland_enhanced.jpg
https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcR1L7Vp40uY29e37HL9sT2lZRkkQEzD6vfp9g&usqp=CAU
https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQXVT1Y9YaeULHEVhjc73Z99R8fBSeSdkyltQ&usqp=CAU
https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQ86eULPeXfYjJaZkS

Found: 100 search results. Extracting links from 0:100
Found: 0 image links, looking for more ...
Found: 0 image links, looking for more ...
Found: 0 image links, looking for more ...
Found: 1 image links, looking for more ...
Found: 1 image links, looking for more ...
Found: 1 image links, looking for more ...
Found: 1 image links, looking for more ...
Found: 1 image links, looking for more ...
Found: 1 image links, looking for more ...
Found: 1 image links, looking for more ...
Found: 1 image links, looking for more ...
Found: 2 image links, looking for more ...
Found: 3 image links, looking for more ...
Found: 4 image links, looking for more ...
Found: 5 image links, looking for more ...
Found: 6 image links, looking for more ...
Found: 6 image links, looking for more ...
Found: 7 image links, looking for more ...
Found: 7 image links, looking for more ...
Found: 8 image links, looking for more ...
Found: 9 image links, looking for more ...
Found: 10 image links, looking for more ..

In [62]:
urls

In [110]:
with open('urls.txt', 'w') as url_file:
    for key in urls.keys():
        counter = 1
        url_file.write('Night\n' if key == 0 else 'Day\n')
        for line in urls[key]:
            print(line)
            url_file.write(f"{counter}- {line} \n")
            counter += 1

https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQ2KhO41jtLlXHtz3aDV4suZ2MjUcY6ru4bUA&usqp=CAU
https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTfu13jBlRQC7cfkqIADaobXGfhMeKU8rOdvg&usqp=CAU
https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSv7Buwi_JW7_yG63iFNDaWzy6ghvWUB-bJYA&usqp=CAU
https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSaubSaVFzwcUavko_bAvITacbBfTdCcXDC9Q&usqp=CAU
https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSfYSemNVNHdCFHyIms98E9ING0o4EDodP2jw&usqp=CAU
https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQL5rTC-aa79Xnn4HkvBfFMb7hqgLm8v5H0QA&usqp=CAU
https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRqYh3oOrt482B4Ffglzv88kC6DgHp0-be6Qw&usqp=CAU
https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQIawi3a0GjiD2Gw65WUBSCBQYAIOAsLpIzpQ&usqp=CAU
https://norwaytravelguide.imgix.net/195136/x/0/?w=860&h=480&fit=crop&auto=format%2C%20compress&dpr=2&ixlib=react-8.6.4
https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSkBxUU6GlUcnrok698AOoQboVIN

In [57]:
map_ = {0: NIGHT_IMGS, 1: DAY_IMGS}
with open('urls.txt', 'r') as url_file:
    lines = url_file.readlines()
    
lines

['Night\n',
 '1- https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQ2KhO41jtLlXHtz3aDV4suZ2MjUcY6ru4bUA&usqp=CAU \n',
 '2- https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTfu13jBlRQC7cfkqIADaobXGfhMeKU8rOdvg&usqp=CAU \n',
 '3- https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSv7Buwi_JW7_yG63iFNDaWzy6ghvWUB-bJYA&usqp=CAU \n',
 '4- https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSaubSaVFzwcUavko_bAvITacbBfTdCcXDC9Q&usqp=CAU \n',
 '5- https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSfYSemNVNHdCFHyIms98E9ING0o4EDodP2jw&usqp=CAU \n',
 '6- https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQL5rTC-aa79Xnn4HkvBfFMb7hqgLm8v5H0QA&usqp=CAU \n',
 '7- https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRqYh3oOrt482B4Ffglzv88kC6DgHp0-be6Qw&usqp=CAU \n',
 '8- https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQIawi3a0GjiD2Gw65WUBSCBQYAIOAsLpIzpQ&usqp=CAU \n',
 '9- https://norwaytravelguide.imgix.net/195136/x/0/?w=860&h=480&fit=crop&auto=format%2C%20compress&dpr=2&ix

In [61]:
for line in lines:
    line = line.strip()
    try:
        if line == 'Night':
            path = map_[0]
            continue

        elif line == 'Day':
            path = map_[1]
            continue

        else:
            line = line.split(' ')[1]    
            scrape.download_image(path, line)
    except:
        print("Skipped line") 

Skipped line
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQ2KhO41jtLlXHtz3aDV4suZ2MjUcY6ru4bUA&usqp=CAU - as /home/ayman/FOE-Linux/Task/Day-Night-Classifier/scraped_dataset/day/368dcab8db.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTfu13jBlRQC7cfkqIADaobXGfhMeKU8rOdvg&usqp=CAU - as /home/ayman/FOE-Linux/Task/Day-Night-Classifier/scraped_dataset/day/5ac6a07a4d.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSv7Buwi_JW7_yG63iFNDaWzy6ghvWUB-bJYA&usqp=CAU - as /home/ayman/FOE-Linux/Task/Day-Night-Classifier/scraped_dataset/day/eed288fcd1.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSaubSaVFzwcUavko_bAvITacbBfTdCcXDC9Q&usqp=CAU - as /home/ayman/FOE-Linux/Task/Day-Night-Classifier/scraped_dataset/day/c1c1b785e0.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSfYSemNVNHdCFHyIms98E9ING0o4EDodP2jw&usqp=CAU - as /home/ayman/FOE-Linux/Task/Day-Night-Classifier/sc

SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRheVCB8DW_hEs8J2eIQuVZEDR6Pq9eYjj4Lg&usqp=CAU - as /home/ayman/FOE-Linux/Task/Day-Night-Classifier/scraped_dataset/day/b3f3871852.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQOq-MJtBNSoU0L13xbIFfxUlTwm9PJwkwtCQ&usqp=CAU - as /home/ayman/FOE-Linux/Task/Day-Night-Classifier/scraped_dataset/day/2274139761.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTeO8P9oNMCXESJNFbeYQQe-rZ3O428AG351w&usqp=CAU - as /home/ayman/FOE-Linux/Task/Day-Night-Classifier/scraped_dataset/day/b599b44c3b.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTs5cuU2CZpOo4mNbORzQfa6ZWnjei8Nzc95w&usqp=CAU - as /home/ayman/FOE-Linux/Task/Day-Night-Classifier/scraped_dataset/day/6a452b9ffa.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTDFnSguOwmxyn0b78ciXnFp2LrDt-AJImrWQ&usqp=CAU - as /home/ayman/FOE-Linux/Task/Day-Night-Classifier/scraped_dataset

SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQyHMy6EDwZjtUP00ZgA53NNSGMHzEI56j64A&usqp=CAU - as /home/ayman/FOE-Linux/Task/Day-Night-Classifier/scraped_dataset/day/b980826579.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSyHqSzTE-LieB5qpVNK4LHIKoEWwe5PDP40Q&usqp=CAU - as /home/ayman/FOE-Linux/Task/Day-Night-Classifier/scraped_dataset/day/4b4e8681a2.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRenVcmsYCzncaQX_Y3lYXpHDCHGhBAyz3Kog&usqp=CAU - as /home/ayman/FOE-Linux/Task/Day-Night-Classifier/scraped_dataset/day/b57e2275a3.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRyx_TV3v4b50gYaR0s8BbiJDIKozQFikMzvA&usqp=CAU - as /home/ayman/FOE-Linux/Task/Day-Night-Classifier/scraped_dataset/day/aa5cf8540a.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQUCfCDIQgYxjm5MxDsXyJ_hGKT81ZioCc5MQ&usqp=CAU - as /home/ayman/FOE-Linux/Task/Day-Night-Classifier/scraped_dataset

SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQ-7L6O1S8XDmJsnCghFw3c6E7tb8t8EapVog&usqp=CAU - as /home/ayman/FOE-Linux/Task/Day-Night-Classifier/scraped_dataset/day/8d739677c7.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRHItJiXopLp4JIa5TpTiXqwYYUokKKgBH8cA&usqp=CAU - as /home/ayman/FOE-Linux/Task/Day-Night-Classifier/scraped_dataset/day/1844b541bb.jpg
SUCCESS - saved https://thumbs.dreamstime.com/z/lovers-near-eiffel-tower-night-paris-travel-spring-france-lovers-near-eiffel-tower-night-paris-travel-186405644.jpg - as /home/ayman/FOE-Linux/Task/Day-Night-Classifier/scraped_dataset/day/1cb435b796.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSINnPcMUQsDygxtu_HRX_xhsP13A8azGzBYw&usqp=CAU - as /home/ayman/FOE-Linux/Task/Day-Night-Classifier/scraped_dataset/day/b5159a8ab9.jpg
SUCCESS - saved https://elements-video-cover-images-0.imgix.net/files/15b08ff4-2a26-43aa-a018-7f8f20ee947f/inline_image_preview.jpg?auto

SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRUdKlM7XeTHOS8mibwVvKksVVOyKikvx-3-w&usqp=CAU - as /home/ayman/FOE-Linux/Task/Day-Night-Classifier/scraped_dataset/day/7472f84fc8.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSG8BrnnHcBfb44ylMbO1QH0P_ywy-2q8kBpg&usqp=CAU - as /home/ayman/FOE-Linux/Task/Day-Night-Classifier/scraped_dataset/day/be303ca07e.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTdPhLDxyS3M0Fo3XVyx3Mago1xehemB_Ju_w&usqp=CAU - as /home/ayman/FOE-Linux/Task/Day-Night-Classifier/scraped_dataset/day/fc34505949.jpg
SUCCESS - saved https://photos.travellerspoint.com/6217/large_Resim%20088.jpg - as /home/ayman/FOE-Linux/Task/Day-Night-Classifier/scraped_dataset/day/86acea2b5d.jpg
SUCCESS - saved https://st3.depositphotos.com/1686288/16507/i/1600/depositphotos_165072592-stock-photo-night-yerevan-armenia-view-from.jpg - as /home/ayman/FOE-Linux/Task/Day-Night-Classifier/scraped_dataset/day/0fcd6e03dd

...

SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSIc3oIHwLQi6IG1kx2wzaIUrT-Xo-b2UzM-g&usqp=CAU - as /home/ayman/FOE-Linux/Task/Day-Night-Classifier/scraped_dataset/day/e8a696f31f.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRKg641E1_x_PneMXaqFKXHpqZHA5iqEo7F2A&usqp=CAU - as /home/ayman/FOE-Linux/Task/Day-Night-Classifier/scraped_dataset/day/16b4fdca08.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRlr9EOwFiD6A94LcHuuzqSGJHZsVPd0O_1JA&usqp=CAU - as /home/ayman/FOE-Linux/Task/Day-Night-Classifier/scraped_dataset/day/306a439c93.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSB-4PpEIe1Iax_RHrFM9yRPSIggx2N2Cad2Q&usqp=CAU - as /home/ayman/FOE-Linux/Task/Day-Night-Classifier/scraped_dataset/day/5e8f682035.jpg
SUCCESS - saved https://i.pinimg.com/originals/0f/7a/25/0f7a25fc3e4cb18743305f061d4284f0.png - as /home/ayman/FOE-Linux/Task/Day-Night-Classifier/scraped_dataset/day/2b849a41d3.jpg
SUC

SUCCESS - saved https://image.shutterstock.com/z/stock-photo-one-dutch-windmill-over-stripes-of-tulip-flowers-field-in-sunny-day-netherlands-243400564.jpg - as /home/ayman/FOE-Linux/Task/Day-Night-Classifier/scraped_dataset/day/76957c2aa0.jpg
SUCCESS - saved https://comps.canstockphoto.com/typical-dutch-village-zaanstad-in-spring-picture_csp28067016.jpg - as /home/ayman/FOE-Linux/Task/Day-Night-Classifier/scraped_dataset/day/6699b74809.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQYr1yGQMxIQogvpasUEhdI7KO8f6xEa_xlDg&usqp=CAU - as /home/ayman/FOE-Linux/Task/Day-Night-Classifier/scraped_dataset/day/6088161e05.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRcgnqO-c3MBJnoWgPK0mbqS2gFQ4TNuuWpoQ&usqp=CAU - as /home/ayman/FOE-Linux/Task/Day-Night-Classifier/scraped_dataset/day/9508711279.jpg
SUCCESS - saved https://media.istockphoto.com/photos/windmills-and-farm-in-the-netherlands-a-sunny-day-museummolen-picture-id687278552 - as /home/ay

SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcT8LR1yi-C3CPhHNT2UuZqXX9b7fCMBcscvXQ&usqp=CAU - as /home/ayman/FOE-Linux/Task/Day-Night-Classifier/scraped_dataset/day/9ce9f68a95.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQU_TvL8FcE1H2x5uQ9cbOpkWGk6s_e0CDxBQ&usqp=CAU - as /home/ayman/FOE-Linux/Task/Day-Night-Classifier/scraped_dataset/day/459809ef42.jpg
SUCCESS - saved https://images.westend61.de/0001258242i/castle-beach-and-st-catherines-island-on-a-sunny-day-in-summer-tenby-pembrokeshire-wales-united-kingdom-europe-RHPLF11565.jpg - as /home/ayman/FOE-Linux/Task/Day-Night-Classifier/scraped_dataset/day/eb8a330e44.jpg
SUCCESS - saved https://as1.ftcdn.net/jpg/01/59/22/88/500_F_159228823_IghJawAdfwS2rxkoCfSkWReYzm7o8L3G.jpg - as /home/ayman/FOE-Linux/Task/Day-Night-Classifier/scraped_dataset/day/cdabc0bb61.jpg
SUCCESS - saved https://thumbs.dreamstime.com/t/big-ben-house-parliament-sunny-day-london-united-kingdom-may-uk-72580397.jpg - 

SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQG_O4gwg7atvG22XZ3oH6NQMI7k95St9GZ8g&usqp=CAU - as /home/ayman/FOE-Linux/Task/Day-Night-Classifier/scraped_dataset/day/2ee351819a.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQ9LTBa5EQ6yV3D96E0fl4IfKWWK84b9_i5fQ&usqp=CAU - as /home/ayman/FOE-Linux/Task/Day-Night-Classifier/scraped_dataset/day/524418e257.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTEPR90wSHa9D2-lJ1iT5QBHxQeU-3iOLiPzA&usqp=CAU - as /home/ayman/FOE-Linux/Task/Day-Night-Classifier/scraped_dataset/day/3931901ec3.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTtmDNNNXND4IbzEoX3H92NZA6yhIQXK59-Ig&usqp=CAU - as /home/ayman/FOE-Linux/Task/Day-Night-Classifier/scraped_dataset/day/21cbe119d1.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTwZJCyAWVNzP0SDiH1ZEIUq_z7DY1v5Lwezg&usqp=CAU - as /home/ayman/FOE-Linux/Task/Day-Night-Classifier/scraped_dataset

SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQcXPez8qrx7-YlDgHFiS_WyywS4lIBKvtgIg&usqp=CAU - as /home/ayman/FOE-Linux/Task/Day-Night-Classifier/scraped_dataset/day/89c4e01659.jpg


In [18]:
wd.quit()